# An introduction to explainability in ML

In [4]:
import sklearn

# Exercise 1: Linear models [20 mins]

In this exercise, we will inspect linear models and their explainability properties.

Write a function that takes in a training dataset consisting of features and labales, and returns the accuracy and coefficients (w and b) of a Logistic Regression model.

Inspect the coefficients of this model using for (1) COMPAS data and (2) The ACS PUMS data and (3) a custom dataset generated for yo below. We used both of these datasets in the previous exercise. Split them into train test sets.

List the top-5 features for each dataset along with its magnitude.

In [92]:
def custom_data():
    x, y = sklearn.datasets.make_classification(n_samples=200, n_features=10, n_informative=2, n_redundant=8, n_repeated=0, random_state=710)
    return x, y
x, y = custom_data()

In [ ]:
# Your code here
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


def train_logistic_regression(x, y):
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(train_x, train_y)
    accuracy = model.score(test_x, test_y)
    return accuracy, (model.coef_[0].tolist())

In [125]:
# inspect the model's coefficients on different datsets
from typing import Optional
import numpy as np
from copy import deepcopy

def inspect_model(x, y, names, leave_out:Optional[list[str]]=None):
    x = deepcopy(x)
    y = deepcopy(y)
    names = deepcopy(names)
    if leave_out:
        for l in leave_out:
            index = names.index(l)
            x = np.delete(x, index, axis=1)
            names.remove(l)
            

    accuracy, coefficients = train_logistic_regression(x, y)
    print(f"Accuracy: {accuracy}")
    # sort by absolute value of coefficients
    sorted_coefficients, sorted_names = zip(*sorted(zip(coefficients, names), key=lambda x: abs(x[0]), reverse=True))
    print(f"Top 5 coefficients: {sorted_coefficients[:5]}")   
    for i in range(min(5, len(sorted_coefficients)-1)):
        print(f"{i+1:>3}. {sorted_names[i]:<20} {sorted_coefficients[i]:>10.2f}")



In [126]:
# get data from COMPAS
from pathlib import Path

import matplotlib.style
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import requests
import folktables
from folktables import ACSDataSource, ACSEmployment


# Selecting the font size here will affect all the figures in this notebook
# Alternatively, you can set the font size for axis labels of each figure separately
font = {'size': 16}
matplotlib.rc('font', **font)

matplotlib.style.use("fivethirtyeight")

# Based on: https://fairlens.readthedocs.io/en/latest/user_guide/compas.html
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
local_name = Path("compas-scores-two-years.csv")
if not local_name.is_file():
    response = requests.get(url)
    with open("compas-scores-two-years.csv", "w") as f:
        f.write(response.content.decode("utf-8"))
df = pd.read_csv(local_name)
df = df.sample(frac=1, random_state=1)

df = df[(df["days_b_screening_arrest"] <= 30)
        & (df["days_b_screening_arrest"] >= -30)
        & (df["is_recid"] != -1)
        & (df["c_charge_degree"] != 'O')
        & (df["score_text"] != 'N/A')].reset_index(drop=True)

In [127]:
import pandas as pd
COMAPS_X = df[["age", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree"]]
COMPAS_X = pd.get_dummies(COMAPS_X, columns=["c_charge_degree"], drop_first=True)
COMPAS_Y = df[["two_year_recid"]]
COMPAS_features = list(COMPAS_X.columns)

In [128]:
from folktables import ACSDataSource, ACSEmployment

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)  # Limiting to AL. You can try another state or all the states.
ACS_X, ACS_Y, _ = ACSEmployment.df_to_numpy(acs_data) 
ACS_features = ACSEmployment.features

In [129]:
print("first dataset")
inspect_model(x, y, list(range(10)))
print("compas dataset")
inspect_model(COMPAS_X, COMPAS_Y, COMPAS_features)
print("acs dataset")
inspect_model(ACS_X, ACS_Y, ACS_features)

first dataset
Accuracy: 0.875
Top 5 coefficients: (1.1455169983026234, -0.9576455909574569, 0.7428626239114834, 0.7276404318201084, -0.4276766685711763)
  1. 6                          1.15
  2. 5                         -0.96
  3. 0                          0.74
  4. 4                          0.73
  5. 8                         -0.43
compas dataset
Accuracy: 0.6874493927125506
Top 5 coefficients: (0.31090441395338253, -0.1729504978355979, 0.16483997605068387, 0.08149657242531187, -0.04067046489506114)
  1. juv_other_count            0.31
  2. c_charge_degree_M         -0.17
  3. priors_count               0.16
  4. juv_fel_count              0.08
  5. age                       -0.04
acs dataset


c:\Users\frede\miniconda3\envs\RAI\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.780975303474257
Top 5 coefficients: (1.333765333306379, 0.7820454942017775, -0.7689995979370305, 0.6256658453247685, -0.48519871960513367)
  1. DIS                        1.33
  2. DREM                       0.78
  3. SEX                       -0.77
  4. MIL                        0.63
  5. DEAR                      -0.49


# Exercise 2: Feature processing [10 mins]

Scale your datasets using the sklean StandardScaler. Retrain the models and show the top-5 features again. Do you notice any differences?

In [ ]:
# Your code here
from sklearn.preprocessing import StandardScaler
# apply standard scaler to the custom data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
print("scaled custom data")
inspect_model(x_scaled, y, list(range(10)))
print("scaled compas data")
COMPAS_X_scaled = scaler.fit_transform(COMPAS_X)
inspect_model(COMPAS_X_scaled, COMPAS_Y, COMPAS_features)

print("scaled acs data")
ACS_X_scaled = scaler.fit_transform(explained_ACS_x)
inspect_model(explained_ACS_x_scaled, ACS_Y, ACS_features)


scaled custom data
Accuracy: 0.875
Top 5 coefficients: (-0.8510715190391881, 0.8510346675067891, 0.7365972359122493, 0.7044431889593377, -0.6647979111774699)
  1. 5                         -0.85
  2. 6                          0.85
  3. 4                          0.74
  4. 0                          0.70
  5. 3                         -0.66
scaled compas data
Accuracy: 0.6874493927125506
Top 5 coefficients: (0.7801647873742068, -0.47599298879161994, 0.14717626113208337, -0.0832975237681818, 0.038508165851969214)
  1. priors_count               0.78
  2. age                       -0.48
  3. juv_other_count            0.15
  4. c_charge_degree_M         -0.08
  5. juv_fel_count              0.04
scaled acs data
Accuracy: 0.7810799497697781
Top 5 coefficients: (-1.1474610770895495, 0.9754147973283497, 0.9209910807235978, -0.8072456149992863, 0.5228099631627076)
  1. AGEP                      -1.15
  2. MIL                        0.98
  3. SCHL                       0.92
  4. ESP          

c:\Users\frede\miniconda3\envs\RAI\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Exercise 3: Removing important features [20 mins]

Remove the top-10% of the features (according to their feature imporrance score) from each training data. Retrain the model again. How much difference do you notice in the accuracy?


In [131]:
# inspect the model's coefficients on different datsets
inspect_model(x_scaled, y, list(range(10)), leave_out=[5, 6])
inspect_model(COMPAS_X_scaled, COMPAS_Y, COMPAS_features, leave_out=["priors_count", "age"])
inspect_model(ACS_X_scaled, ACS_Y, ACS_features, leave_out = ["AGEP", "MIL"])

Accuracy: 0.875
Top 5 coefficients: (1.215169413658818, 1.1579733757980806, -1.0880055959621193, -0.8112489650656605, -0.7235119601958268)
  1. 4                          1.22
  2. 0                          1.16
  3. 3                         -1.09
  4. 8                         -0.81
  5. 9                         -0.72


c:\Users\frede\miniconda3\envs\RAI\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.5838056680161944
Top 5 coefficients: (0.24896520176010467, 0.23671551642325003, 0.23046452661386355, -0.19679044420880423)
  1. juv_other_count            0.25
  2. juv_misd_count             0.24
  3. juv_fel_count              0.23
Accuracy: 0.7445583926329008
Top 5 coefficients: (-1.1268300751062796, 1.0073742779157528, 0.6803894540637497, -0.29244090065488615, 0.27272942029477565)
  1. ESP                       -1.13
  2. SCHL                       1.01
  3. DIS                        0.68
  4. RELP                      -0.29
  5. DREM                       0.27


# Exercise 4: Decision trees [15 mins]

Train decision trees of depth 1, 5, 10 and 20. Visualize the trees using [plot_tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html). What difference do you see in accuracy and the actual trees?

Retrain the trees for different train/test splits. Do you see any difference in the trees?

In [ ]:
# Your code here.

# Exercise 5: Generating SHAP attributions [15 mins]

Take the ACS PUMS data without adding any feature scaling or normalization, and train the following two models:
1. A sklearn neural network with hidden units [100, 100].
2. A Random Forest classifier.

Compute the feature attributions using the [SHAP library](https://github.com/shap/shap). How do the feature attributions of the two models compare?

In [ ]:
# Your code here
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_neural_network(x, y, scaler=None):
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
    if scaler:
        model = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42))
    else:
        model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
    model.fit(train_x, train_y)
    predictions = model.predict(test_x)
    accuracy = accuracy_score(test_y, predictions)
    return accuracy, model

ACS_X, ACS_Y, ACS_features = ACSEmployment.df_to_numpy(acs_data)



In [143]:
accuracy, ACS_model = train_neural_network(ACS_X, ACS_Y)
print(f"Accuracy: {accuracy}")

Accuracy: 0.81802009208874


In [158]:
from shap import KernelExplainer as Explainer
import shap
import matplotlib.pyplot as plt

# Create a SHAP explainer
explained_ACS_x = np.array([ACS_X[:10]])
explainer = Explainer(ACS_model.predict_proba, explained_ACS_x)
shap_values = explainer(explained_ACS_x)
# Plot the SHAP values
shap.summary_plot(shap_values, explained_ACS_x, feature_names=ACS_features)
# Plot the SHAP values for a single prediction
shap.initjs()
shap.plots.waterfall(shap_values[0], max_display=10)
# Plot the SHAP values for a single prediction
shap.plots.force(shap_values[0], matplotlib=True)
# Plot the SHAP values for a single prediction
shap.plots.bar(shap_values[0], max_display=10)
# Plot the SHAP values for a single prediction
shap.plots.decision(shap_values[0], matplotlib=True)
# Plot the SHAP values for a single prediction
shap.plots.text(shap_values[0])
# Plot the SHAP values for a single prediction
shap.plots.image(shap_values[0])


Provided model function fails when applied to the provided data set.


ValueError: Found array with dim 3. MLPClassifier expected <= 2.

# Exercise 6: Effect of baseline on SHAP values [30 mins]

For each model, try out three different baselines:
1. All zeros
2. Mean
3. Median

How do the SHAP values compare between the three baselines?

Now normalize the data using the StandardScaler and repeat the step above. Do you see any differences?

In [ ]:
# Your code here